<h1 style="background-color:#FF7733;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🌐 תהליך הסבת חלקות (דו-ממדיות) בתהליך לבנק"ל מודרני 🌐

In [ ]:
print('14. Parcels Borders In-Process:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import datetime as dt
import pandas as pd
import geopandas as gpd
import subprocess

<h1 style="text-align:right;font-size:100%">
העתקת שכבת מקור והוספת שזה זיהוי זמני לצורך איחוד בסוף הסבה

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\Parcels2D_InProcessp-pre_calc.bat"]);

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  14.a Reading inputs')

In [ ]:
Results_gdb = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\Results.gdb'

today =  dt.date.today()
product_name =  str(today.year) + str(today.month) + str(today.day)  + '_ncdb.gdb'
product_path = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים/' + product_name
del [today, product_name]

<h1 style="text-align:right;font-size:100%">
שכבת חלקות בתהליך קיימת

In [ ]:
Parcels_InProcess_cols = ['TEMP_ID', 'PARCELNAME', 'GUSHNUM', 'GUSHSUFFIX', 'TALAR_NUM', 'TALAR_YEAR', 'CADASTER_PROCESS', 'ENTITY_STATE', 'TABAPLAN', 'LEGALAREA']

Parcels_InProcess = gpd.read_file(filename        = Results_gdb,
                                  layer           = 'CADSDE_Parcels_InProcess',
                                  include_fields  = Parcels_InProcess_cols,
                                  ignore_geometry = True)


del [Results_gdb, Parcels_InProcess_cols]

<h1 style="text-align:right;font-size:100%">
שכבת גושי קדסטר מודרנית

In [ ]:
Blocks_cols = ['BlockUniqueID', 'BlockNumber', 'SubBlockNumber']

Blocks = gpd.read_file(product_path,
                       layer = 'Blocks',
                       include_fields = Blocks_cols,
                       ignore_geometry = True)


del Blocks_cols

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תהליכי קדסטר מודרנית

In [ ]:
cpb_cols = ['CPBUniqueID', 'ProcessName', 'ProcessType']

cpb_layer = gpd.read_file(product_path,
                          layer = 'CadasterProcessBorders',
                          include_fields = cpb_cols,
                          ignore_geometry = True)


del cpb_cols

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  14.b Cleaning data')

<h1 style="text-align:right;font-size:125%;">
:עיבוד נתוני חלקות בתהליך

<h1 style="text-align:right;font-size:100%">
הסרת רשומות כפולות

In [ ]:
Parcels_InProcess.drop_duplicates(subset  = Parcels_InProcess.columns,
                                  keep    = 'first',
                                  inplace = True)

<h1 style="text-align:right;font-size:100%">
:הסרת חלקות רקע או חלקות לא ידועות
<h1 style="text-align:right;font-size:100%">
שמירת חלקות ביסוס (0) חדשות (1) 
<h1 style="text-align:right;font-size:100%">
(NaN) והשמטת חלקות רקע (2) ולא ידועות
<h1 style="text-align:right;font-size:100%">
הוספת 1 לערכים על מנת שערכים יהיו חלקות ביסוס: 1, חלקות חדשות: 2

In [ ]:
Parcels_InProcess = Parcels_InProcess[Parcels_InProcess['ENTITY_STATE'].isin([0, 1])]
Parcels_InProcess.dropna(subset = ['ENTITY_STATE'], inplace = True)
Parcels_InProcess['ENTITY_STATE'] = Parcels_InProcess['ENTITY_STATE'] + 1
Parcels_InProcess['ENTITY_STATE'] = Parcels_InProcess['ENTITY_STATE'].astype(int)

<h1 style="text-align:right;font-size:100%">
:פיצול שדה של שם חלקה לשתי שדות
<h1 style="text-align:right;font-size:100%">
סוג חלקה - 
<h1 style="text-align:right;font-size:100%">
<pre class="tab">     ארעית = 1 = T   </pre> 
<h1 style="text-align:right;font-size:100%">
<pre class="tab">      סופית = 2 = F   </pre>
<h1 style="text-align:right;font-size:100%">
<pre class="tab">  חלקות ללא קידומת מקורן מתת"ג או קמ"ק ולכן יקבלו את הקידומת כחלקות סופיות (2)</pre>  
<h1 style="text-align:right;font-size:100%">
מספר חלקה - 

In [ ]:
Parcels_InProcess['ParcelType'] = Parcels_InProcess['PARCELNAME'].str.get(0)
Parcels_InProcess['ParcelType'] = Parcels_InProcess['ParcelType'].astype(str)


unknown_condition = ~Parcels_InProcess['ParcelType'].isin(['T', 'F'])
Parcels_InProcess['ParcelType'] = np.where(unknown_condition,
                                           'F',
                                           Parcels_InProcess['ParcelType'])

del unknown_condition


ParcelType_clf = {'T' : 1, 'F' : 2}
Parcels_InProcess['ParcelType'] = Parcels_InProcess['ParcelType'].map(ParcelType_clf)

In [ ]:
Parcels_InProcess['ParcelNumber'] = Parcels_InProcess['PARCELNAME'].map(lambda x: x.lstrip('FT'))
Parcels_InProcess['ParcelNumber'] = Parcels_InProcess['ParcelNumber'].str.partition(":")[0]
Parcels_InProcess['ParcelNumber'] = Parcels_InProcess['ParcelNumber'].astype(int)

<h1 style="text-align:right;font-size:100%">
המרת יחידת שטח רשום מדונם אל מטר רבוע

In [ ]:
Parcels_InProcess['LEGALAREA'] = Parcels_InProcess['LEGALAREA'] * 1000

<h1 style="text-align:right;font-size:100%">
הסרת שדה לא רלוונטי

In [ ]:
Parcels_InProcess.drop(columns='PARCELNAME', inplace=True)

<h1 style="text-align:right;font-size:100%">
שינוי שמות של שדות

In [ ]:
Parcels_InProcess.rename(columns = {'CADASTER_PROCESS' : 'ProcessType',
                                    'TABAPLAN'         : 'LandDesignationPlan',
                                    'ENTITY_STATE'     : 'ParcelRole',
                                    'LEGALAREA'        : 'LegalArea'},
                         inplace = True)

<h1 style="text-align:right;font-size:100%">
מיפוי ערכי סוג תהליך יוצר/מעדכן

In [ ]:
ProcessType_clf = {1  : 9,  #הסדר מקרקעין 
                   2  : 1,  #תכנית לצרכי רישום
                   8  : 11, #תצ"ר בשטח לא מוסדר
                   9  : 5, #קדסטר מבוסס קואורדינטות
                   12 : 4} #תשריט לתיעוד גבולות

Parcels_InProcess['ProcessType'] = Parcels_InProcess['ProcessType'].map(ProcessType_clf)
Parcels_InProcess['ProcessType'] = Parcels_InProcess['ProcessType'].astype('Int64')

<h1 style="text-align:right;font-size:100%">
יצירת שדה מקשר למספר גוש מלא ומיזוג עם שכבת גושי קדסטר מודרנית לקבלת מספר מזהה של גוש

In [ ]:
Blocks['BlockName'] = Blocks['BlockNumber'].astype(str) + '_' + Blocks['SubBlockNumber'].astype(str)

Parcels_InProcess['BlockName'] = Parcels_InProcess['GUSHNUM'].astype(str) + '_' + Parcels_InProcess['GUSHSUFFIX'].astype(str)


Parcels_InProcess = Parcels_InProcess.merge(Blocks[['BlockUniqueID', 'BlockName']], on='BlockName', how='left')
Parcels_InProcess.drop(columns = ['GUSHNUM', 'GUSHSUFFIX', 'BlockName'], inplace = True)

del Blocks

<h1 style="text-align:right;font-size:100%">
יצירת שדה מקשר למספר תהליך מלא (אליפסה/מלבן) ומיזוג עם שכבת גבולות תהליכי קדסטר מודרנית

In [ ]:
Parcels_InProcess['TALAR_NUM'] = Parcels_InProcess['TALAR_NUM'].fillna(0)
Parcels_InProcess['TALAR_NUM'] = Parcels_InProcess['TALAR_NUM'].astype(int)

Parcels_InProcess['TALAR_YEAR'] = Parcels_InProcess['TALAR_YEAR'].fillna(0)
Parcels_InProcess['TALAR_YEAR'] = Parcels_InProcess['TALAR_YEAR'].astype(int)

Parcels_InProcess['ProcessName'] = Parcels_InProcess['TALAR_NUM'].astype(str) + '/' + Parcels_InProcess['TALAR_YEAR'].astype(str)

<h1 style="text-align:right;font-size:100%">
סינון שכבת גבולות תהליכי קדסטר לסוגי תהליך של תצ"ר בלבד

In [ ]:
cpb_layer = cpb_layer[cpb_layer['ProcessType'] == 1]
cpb_layer.drop(columns='ProcessType', inplace=True)

<h1 style="text-align:right;font-size:100%">
איחוד חלקות בתהליך עם גבולות תצ"ר לפי שם המפה, לצורך קבלת שדה מספר מזהה של התהליך

In [ ]:
Parcels_InProcess = Parcels_InProcess.merge(cpb_layer, on='ProcessName', how='left')

Parcels_InProcess.drop(columns=['TALAR_NUM', 'TALAR_YEAR'], inplace=True)


del cpb_layer

<h1 style="text-align:right;font-size:100%">
הגדרת שדות זיהוי של תהליך יוצר, תהליך מעדכן ותהליך מבטל

In [ ]:
CreateProcess_condition = Parcels_InProcess['ProcessType'].isin([1, 9, 11])
Parcels_InProcess['CPBUniqueCreateProcessID'] = np.where(CreateProcess_condition,
                                                         Parcels_InProcess['CPBUniqueID'], 0)
Parcels_InProcess['CPBUniqueCreateProcessID'] = Parcels_InProcess['CPBUniqueCreateProcessID'].fillna(0)


UpdateProcess_condition = Parcels_InProcess['ProcessType'].isin([4, 5])
Parcels_InProcess['CPBUniqueUpdateProcessID'] = np.where(UpdateProcess_condition,
                                                         Parcels_InProcess['CPBUniqueID'], 0)
Parcels_InProcess['CPBUniqueUpdateProcessID'] = Parcels_InProcess['CPBUniqueUpdateProcessID'].fillna(0)

Parcels_InProcess['CPBUniqueCancelProcessID'] = 0


del [CreateProcess_condition, UpdateProcess_condition]
Parcels_InProcess.drop(columns = 'CPBUniqueID', inplace=True)

<h1 style="text-align:right;font-size:100%">
הסרת כפילויות

In [ ]:
Parcels_InProcess = Parcels_InProcess.sort_values(['ParcelNumber', 'BlockUniqueID', 'ProcessType', 'ParcelType','ParcelRole'])

Parcels_InProcess.drop_duplicates(subset=['ParcelNumber', 'BlockUniqueID', 'CPBUniqueCreateProcessID', 'ParcelType','ParcelRole', 'ProcessType'], keep = 'last', inplace = True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  14.c Data conversion')

In [ ]:
new_columns = np.dtype([
                        ("ParcelUniqueID", int),
                        ("ParcelNumber", int),
                        ("ParcelType", int),
                        ("ParcelRole", int),
                        ("CreateProcessType", int),
                        ("LegalArea", float),
                        ("LandDesignationPlan", str),
                        ("BlockUniqueID", int),
                        ("CPBUniqueCreateProcessID", int),
                        ("CPBUniqueUpdateProcessID", int),
                        ("CPBUniqueCancelProcessID", int),
                        ])


InProcessParcels2D = pd.DataFrame(np.empty(0, dtype=new_columns))

del new_columns

<h1 style="text-align:right;font-size:100%">
איכלוס שדות

In [ ]:
InProcessParcels2D['TEMP_ID'] = Parcels_InProcess['TEMP_ID'].astype('Int64')


InProcessParcels2D['ParcelNumber'] = Parcels_InProcess['ParcelNumber'].astype('Int64')


InProcessParcels2D['ParcelType'] = Parcels_InProcess['ParcelType'].astype('Int64')


InProcessParcels2D['ParcelRole'] = Parcels_InProcess['ParcelRole'].astype('Int64')


InProcessParcels2D['CreateProcessType'] = Parcels_InProcess['ProcessType'].astype('Int64')


InProcessParcels2D['LegalArea'] = Parcels_InProcess['LegalArea'].astype(float)


InProcessParcels2D['LandDesignationPlan'] = Parcels_InProcess['LandDesignationPlan'].astype(str)


InProcessParcels2D['BlockUniqueID'] = Parcels_InProcess['BlockUniqueID'].astype('Int64')


InProcessParcels2D['CPBUniqueCreateProcessID'] = Parcels_InProcess['CPBUniqueCreateProcessID'].astype('Int64')


InProcessParcels2D['CPBUniqueUpdateProcessID'] = Parcels_InProcess['CPBUniqueUpdateProcessID'].astype('Int64')


InProcessParcels2D['CPBUniqueCancelProcessID'] = Parcels_InProcess['CPBUniqueCancelProcessID'].astype('Int64')


InProcessParcels2D['ParcelUniqueID'] = np.arange(1, len(Parcels_InProcess) + 1)


del Parcels_InProcess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  14.e Exporting results to product folder')

In [ ]:
xlsx_dir  = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\InProcessParcels2D_data.xlsx'

InProcessParcels2D.to_excel(xlsx_dir, encoding='cp1255', index=False)

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\Parcels2D In-Process-Save as Feature Class.bat"]);